## Homework: Vector Search

In this homework, we'll experiment with vector with and without Elasticsearch

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Q1. Getting the embeddings model

First, we will get the embeddings model `multi-qa-distilbert-cos-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

```python
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embedding for this user question:

```python
user_question = "I just discovered the course. Can I still join it?"
```

What's the first value of the resulting vector?


* 0.07

Q1: 0.07822264

## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`:

- Create a list `embeddings` 
- Iterate over each document 
- `qa_text = f'{question} {text}'`
- compute the embedding for `qa_text`, append to `embeddings`
- At the end, let `X = np.array(embeddings)` (`import numpy as np`)

What's the shape of X? (`X.shape`). Include the parantheses.  

Q2: (375, 768)

## Q3. Search

We have the embeddings and the query vector. Now let's compute the 
cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2. 

The vectors returned from the embedding model are already
normalized (you can check it by computing a dot product of a vector
with itself - it should return 1.0). This means that in order
to compute the coside similarity, it's sufficient to 
multiply the matrix `X` by the vector `v`:

```python
scores = X.dot(v)
```

What's the highest score in the results?

- 65.0 
- 6.5
- 0.65
- 0.065

Q3: 0.6506574

## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will
use the hitrate metric for evaluation.

First, load the ground truth dataset:

```python
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
```

Now use the code from the module to calculate the hitrate of
`VectorSearchEngine` with `num_results=5`.

What did you get?

* 0.93
* 0.73
* 0.53
* 0.33

Q4: 0.9398907103825137

## Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

* Create the index with the same settings as in the module (but change the dimensions)
* Index the embeddings (note: you've already computed them)

After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

## Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between
the query and ALL the vectors in our database. Usually this is 
not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster. 

Let's evaluate how worse the results are when we switch from
exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

* 0.93
* 0.73
* 0.53
* 0.33

Q6: 0.9398907103825137

## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw3
* It's possible that your answers won't match exactly. If it's the case, select the closest one.


## Answer Q1

In [2]:
from sentence_transformers import SentenceTransformer
# embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')
# embedding_model.save('./model/multi-qa-distilbert-cos-v1')

In [5]:
# Загрузка модели из локальной папки
embedding_model = SentenceTransformer('./model/multi-qa-distilbert-cos-v1')
user_question = "I just discovered the course. Can I still join it?"
user_question_embedding = embedding_model.encode(user_question)
user_question_embedding[0]

0.07822264

## Answer Q2

In [6]:
import requests
import numpy as np

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

# Filtering to get only 375 documents for "machine-learning-zoomcamp"
filtered_documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']
documents = filtered_documents[:375]

embeddings = []
for doc in documents:
    qa_text = f"{doc['question']} {doc['text']}"
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

X = np.array(embeddings)
X.shape

(375, 768)

## Answer Q3

In [7]:
v = user_question_embedding
scores = X.dot(v)
scores.max()

0.6506574

## Answer Q4

In [8]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
print(ground_truth[0])

{'question': 'Where can I sign up for the course?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}


In [9]:
class VectorSearchEngine:
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

# Ініціалізація пошукової системи з векторами документів
search_engine = VectorSearchEngine(documents=documents, embeddings=X)

# Функція для оцінки точності
def calculate_hitrate(search_engine, ground_truth, num_results=5):
    hits = 0
    for item in ground_truth:
        query = item['question']
        true_doc_id = item['document']
        query_embedding = embedding_model.encode(query)
        results = search_engine.search(query_embedding, num_results=num_results)
        result_ids = [doc['id'] for doc in results]
        if true_doc_id in result_ids:
            hits += 1
    return hits / len(ground_truth)

hitrate = calculate_hitrate(search_engine, ground_truth, num_results=5)
print(f'Hitrate: {hitrate}')

Hitrate: 0.9398907103825137


## Answer Q5

In [10]:
from elasticsearch import Elasticsearch, helpers
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '41163b8eb97a', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'kRqKw1mDSbWdAjPPq0GtyQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [51]:
# Створення індексу з відповідними налаштуваннями
index_name = 'machine-learning-zoomcamp'
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "text"},
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [52]:
# Видалення індексу, якщо він існує
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)

In [53]:
# Створення нового індексу
es_client.indices.create(index=index_name, settings=index_settings['settings'], mappings=index_settings['mappings'])

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'machine-learning-zoomcamp'})

In [54]:
# Індексація документів з обчисленими векторами
def generate_docs():
    for doc, embedding in zip(documents, X):
        yield {
            "_index": index_name,
            "_id": doc["id"],
            "_source": {
                "text_vector": embedding.tolist(),
                "text": doc["text"],
                "question": doc["question"]
            }
        }

helpers.bulk(es_client, generate_docs())

(375, [])

In [55]:
# Пошук за допомогою векторного запиту з питання 1
search_term = "I just discovered the course. Can I still join it?"
vector_search_term = embedding_model.encode(search_term)

query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

# Виконання пошуку
response = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course", "id"])

# Виведення ID документа з найвищим результатом
if response['hits']['total']['value'] > 0:
    highest_score_doc_id = response['hits']['hits'][0]['_id']
    print(f'ID документа з найвищим результатом: {highest_score_doc_id}')
else:
    print('Не знайдено документів за запитом.')

response["hits"]["hits"]

ID документа з найвищим результатом: ee58a693


[{'_index': 'machine-learning-zoomcamp',
  '_id': 'ee58a693',
  '_score': 0.825329,
  '_source': {'question': 'The course has already started. Can I still join it?',
   'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.'}},
 {'_index': 'machine-learning-zoomcamp',
  '_id': '0a278fb2',
  '_score': 0.7358537,
  '_source': {'question': 'I just joined. What should I do next? How can I access course materials?',
   'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos.

## Answer Q6

In [56]:
# Функція для оцінки точності в Elasticsearch
def calculate_hitrate_elasticsearch(ground_truth):
    hits = 0
    for gt_qa in ground_truth:
        query = {
            "field": "text_vector",
            "query_vector": embedding_model.encode(gt_qa['question']),
            "k": 5,
            "num_candidates": 10000, 
        }
        res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course", "id"])
        result_ids = [hit['_id'] for hit in response['hits']['hits']]
        true_doc_id = gt_qa['document']
        if true_doc_id in result_ids:
            hits += 1
    return hits / len(ground_truth)

In [57]:
# Обчислення точності для Elasticsearch
hitrate_elastic = calculate_hitrate_elasticsearch(ground_truth)
print(f'Hitrate для Elasticsearch: {hitrate_elastic}')

Hitrate для Elasticsearch: 0.01366120218579235


In [58]:
len(ground_truth)

1830